In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms

def get_device():
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print(f"Using GPU: {torch.cuda.get_device_name(0)}")
    else:
        device = torch.device("cpu")
        print("Using CPU")
    return device

device = get_device()

Using GPU: NVIDIA GeForce RTX 4090


In [4]:
from npz_loader import discover_games

game_paths = discover_games(Path("dataset/"))
print("\n".join(str(p) for p in game_paths))

dataset\BeamRiderNoFrameskip-v4\BeamRiderNoFrameskip-v4
dataset\BreakoutNoFrameskip-v4\BreakoutNoFrameskip-v4
dataset\EnduroNoFrameskip-v4\EnduroNoFrameskip-v4
dataset\MsPacmanNoFrameskip-v4\MsPacmanNoFrameskip-v4
dataset\PongNoFrameskip-v4\PongNoFrameskip-v4
dataset\QbertNoFrameskip-v4\QbertNoFrameskip-v4
dataset\SeaquestNoFrameskip-v4\SeaquestNoFrameskip-v4
dataset\SpaceInvadersNoFrameskip-v4\SpaceInvadersNoFrameskip-v4


In [5]:
from npz_loader import discover_game_npz_paths

train_game_dirs = [
    Path(r"dataset\BeamRiderNoFrameskip-v4\BeamRiderNoFrameskip-v4"),
    Path(r"dataset\BreakoutNoFrameskip-v4\BreakoutNoFrameskip-v4"),
]

game_npz_paths = discover_game_npz_paths(train_game_dirs)

for key in game_npz_paths:
    print(f"{key}: {len(game_npz_paths[key])} npz files")

dataset\BeamRiderNoFrameskip-v4\BeamRiderNoFrameskip-v4: 200 npz files
dataset\BreakoutNoFrameskip-v4\BreakoutNoFrameskip-v4: 200 npz files


In [22]:
key_one = list(game_npz_paths.keys())[0]
file_one = game_npz_paths[key_one][0]

data = np.load(file_one)

one_row = []
for key in data.keys():
    print(f"{key}: {data[key].shape}")

    if len(data[key]) > 1:
        one_row.append(f"{key}: {data[key][20]}")
    else:
        one_row.append(f"{key}: {data[key][0]}")

print("One row:")
for x in one_row:
    print("\t" + x)

model selected actions: (548, 1)
taken actions: (548, 1)
obs: (548,)
rewards: (548,)
episode_returns: (1,)
episode_starts: (548,)
repeated: (548,)
One row:
	model selected actions: [8]
	taken actions: [8]
	obs: C:\Users\Brian\Documents\Code\Research\IRL\data\rgb_testing_onelife_fullimg\rand_steps=0\DQN_10000000_episode_life=True\BeamRiderNoFrameskip-v4\BeamRiderNoFrameskip-v4-recorded_images-0\20.png
	rewards: 0.0
	episode_returns: 7.0
	episode_starts: False
	repeated: False


In [24]:
from npz_loader import AtariData

data = AtariData(game_npz_paths)

defaultdict(list,
            {WindowsPath('dataset/BeamRiderNoFrameskip-v4/BeamRiderNoFrameskip-v4'): [NpzFile 'dataset\\BeamRiderNoFrameskip-v4\\BeamRiderNoFrameskip-v4\\BeamRiderNoFrameskip-v4_0.npz' with keys: model selected actions, taken actions, obs, rewards, episode_returns...,
              NpzFile 'dataset\\BeamRiderNoFrameskip-v4\\BeamRiderNoFrameskip-v4\\BeamRiderNoFrameskip-v4_1.npz' with keys: model selected actions, taken actions, obs, rewards, episode_returns...,
              NpzFile 'dataset\\BeamRiderNoFrameskip-v4\\BeamRiderNoFrameskip-v4\\BeamRiderNoFrameskip-v4_10.npz' with keys: model selected actions, taken actions, obs, rewards, episode_returns...,
              NpzFile 'dataset\\BeamRiderNoFrameskip-v4\\BeamRiderNoFrameskip-v4\\BeamRiderNoFrameskip-v4_100.npz' with keys: model selected actions, taken actions, obs, rewards, episode_returns...,
              NpzFile 'dataset\\BeamRiderNoFrameskip-v4\\BeamRiderNoFrameskip-v4\\BeamRiderNoFrameskip-v4_101.npz' with